In [ ]:
import os
import phoenix as px

# 將 PHOENIX_COLLECTOR_ENDPOINT 指向本機實際 IP
os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "http://192.168.1.122:6006"

client = px.Client()
dataset = client.get_dataset(
    name="my-chinese-ods-dataset",
    version_id="RGF0YXNldFZlcnNpb246OQ=="
)

/opt/conda/lib/python3.10/site-packages/phoenix/utilities/client.py:60: UserWarning: The Phoenix server (8.12.0) and client (8.27.1) versions are mismatched and may have compatibility issues.
  warnings.warn(


In [ ]:
import phoenix
print(phoenix.__version__)

8.27.1


In [ ]:
print(dataset)

Dataset(id='RGF0YXNldDo3', version_id='RGF0YXNldFZlcnNpb246OQ==')


In [ ]:
print(dir(dataset))

['__annotations__', '__class__', '__dataclass_fields__', '__dataclass_params__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__match_args__', '__module__', '__ne__', '__new__', '__post_init__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_keys', 'as_dataframe', 'examples', 'from_dict', 'id', 'version_id']


In [ ]:
print(dataset.as_dataframe())

                                                 input  \
example_id                                               
RGF0YXNldEV4YW1wbGU6NTA=      {'input': '艾菲爾鐵塔位於哪裡？ '}   
RGF0YXNldEV4YW1wbGU6NTE=       {'input': '中國的長城有多長？ '}   
RGF0YXNldEV4YW1wbGU6NTI=     {'input': '最大的熱帶雨林是什麼？ '}   
RGF0YXNldEV4YW1wbGU6NTM=    {'input': '地球上最高的山是哪一座？ '}   
RGF0YXNldEV4YW1wbGU6NTQ=    {'input': '世界上最長的河流是什麼？ '}   
RGF0YXNldEV4YW1wbGU6NTU=       {'input': '蒙娜麗莎是誰畫的？ '}   
RGF0YXNldEV4YW1wbGU6NTY=        {'input': '人體有幾塊骨頭？ '}   
RGF0YXNldEV4YW1wbGU6NTc=  {'input': '太陽系中最大的行星是哪一顆？ '}   
RGF0YXNldEV4YW1wbGU6NTg=  {'input': '《羅密歐與茱麗葉》是誰寫的？ '}   
RGF0YXNldEV4YW1wbGU6NTk=  {'input': '第一次登月是什麼時候發生的？ '}   

                                                                     output  \
example_id                                                                    
RGF0YXNldEV4YW1wbGU6NTA=                        {'output': '艾菲爾鐵塔位於法國巴黎。 '}   
RGF0YXNldEV4YW1wbGU6NTE=       {'output': '中國的長城長約 13,171 英里（21,19

In [ ]:
df = dataset.as_dataframe()
print(df.columns)

Index(['input', 'output', 'metadata'], dtype='object')


In [ ]:
df["input"] = df["input"].apply(lambda x: x['input'])
df["output"] = df["output"].apply(lambda x: x['output'])


In [ ]:
MY_CUSTOM_TEMPLATE = '''
    You are evaluating whether the output mentions any person's name, such as common first names, full names, or references to individuals.
    [BEGIN DATA]
    ************
    [Question]: {input}
    ************
    [Response]: {output}
    [END DATA]


    Please focus on the tone of the response.
    Your answer must be single word, either "yes" or "no"
    '''

In [30]:
import openai
import os
from phoenix.evals import llm_classify, OpenAIModel


rails = ['yes', 'no']

client = openai.OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url=os.getenv("OPENAI_API_BASE")
)

def call_o1_model(prompt: str) -> str:
    response = client.chat.completions.create(
        model="o1",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.6
    )
    return response.choices[0].message["content"]


model = OpenAIModel(model="o1",temperature=0.6)
positive_eval = llm_classify(
    data=df,
    template=MY_CUSTOM_TEMPLATE,
    model=call_o1_model,  
    rails=rails, 
    concurrency=5 
)


AttributeError: 'function' object has no attribute 'reload_client'

In [ ]:
#Phoenix Evals support using either strings or objects as templates
MY_CUSTOM_TEMPLATE = " ..."
MY_CUSTOM_TEMPLATE = PromptTemplate("This is a test {prompt}")

In [ ]:
# Run an experiment
from phoenix.experiments import run_experiment

experiment = run_experiment(dataset, my_task, evaluators=evaluators)